In [4]:
import duckdb
import logging
import pandas as pd
from pathlib import Path
from data.settings import settings
from data.alpha_vantage import AlphaVantageClient
from data.alpha_vantage_schema import ENDPOINT_TO_TABLE_MAP, TABLE_SCHEMAS, DEFAULT_ENDPOINTS

In [ ]:
# Ensure tables exist
db_path = Path(settings.get("data_dir"), settings.get("db_name"))
conn = duckdb.connect(str(db_path), read_only=False)
tables = set()

for endpoint_name in DEFAULT_ENDPOINTS:
   tables.add(ENDPOINT_TO_TABLE_MAP.get(endpoint_name, endpoint_name).upper())

for table_name in tables:
   schema_sql = TABLE_SCHEMAS.get(table_name)
   if schema_sql:
      create_sql = schema_sql.replace("CREATE TABLE", "CREATE TABLE IF NOT EXISTS")
      conn.execute(create_sql)

client = AlphaVantageClient(db_conn=conn)

data = client._fetch_data(
   endpoint_name="HISTORICAL_OPTIONS",
   params={"symbol": "GOOG", "date": "2011-11-24", "datatype": "csv"}
)
print(data)

conn.close()

In [8]:
db_path = Path(settings.get("data_dir"), settings.get("db_name"))
conn = duckdb.connect(str(db_path), read_only=True)

result = conn.execute(
   """
   SELECT *
   FROM MODEL_METADATA
   """).df()

# print(f"Dataset Shape: {result.shape}")
# result.dropna(axis=1, how="all", inplace=True)
# result.ffill(inplace=True)
# result.dropna(axis=0, how="any", inplace=True)

print(result)
# result.to_clipboard()

conn.close()

                          dt model_name model_type  \
0 2025-12-31 05:10:04.379296      model     TITANS   
1 2025-12-31 07:16:55.621068      model     TITANS   
2 2025-12-31 07:19:04.503391      model     TITANS   
3 2025-12-31 07:21:01.243668      model     TITANS   
4 2025-12-31 07:28:38.268827      model     TITANS   
5 2025-12-31 07:28:46.411477      model     TITANS   
6 2025-12-31 07:42:54.418852      model     TITANS   
7 2025-12-31 07:43:16.334821      model     TITANS   
8 2025-12-31 07:57:13.277843      model     TITANS   
9 2025-12-31 08:04:55.285637      model     TITANS   

                        training_run_id  best_val_loss  dataset_size  \
0  3d804de1-f83e-47fd-92e4-2bd1c53acc52       0.036603          1297   
1  11af2962-8f57-412f-b699-07c971c93f71       0.030713          1297   
2  f488c686-f077-4caf-ac71-59d0f3e72640       0.019325          1297   
3  534429e7-b4ad-4c07-9f40-c4796f8564ce       0.049136          1297   
4  a1479984-1f45-4c16-adfc-46b57e7fdb16      

In [ ]:
   """
   WITH fundamental_pivoted AS (
       PIVOT (
           SELECT symbol, dt, metric, value 
           FROM FUNDAMENTALS 
           WHERE periodType = 'QUARTERLY'
       )
       ON metric 
       USING AVG(value) 
       GROUP BY symbol, dt
   )
   SELECT 
       tsda.symbol,
       tsda.dt,
       tsda.* EXCLUDE (symbol, dt),
       over.* EXCLUDE (symbol, dt),
       fp.* EXCLUDE (symbol, dt),
       m.* EXCLUDE (dt)
   FROM TIME_SERIES_DAILY_ADJUSTED tsda
   LEFT JOIN OVERVIEW over 
       ON tsda.symbol = over.symbol
   ASOF LEFT JOIN fundamental_pivoted fp 
       ON tsda.symbol = fp.symbol AND tsda.dt >= fp.dt
   ASOF LEFT JOIN MACRO m 
       ON tsda.dt >= m.dt
   ORDER BY tsda.symbol, tsda.dt
   """